In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt


In [21]:
# device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [22]:
# hyper parameters
num_epochs = 4
batch_size = 32
learning_rate = 3e-4

In [23]:
# dataset has PILImage images of range [0, 1]
# we transform them to tensors with values between (-1, 1)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [24]:
# get datasets and dataloaders

train_dataset = torchvision.datasets.CIFAR10(root = "./data", train = True,
                                            download = True,
                                            transform = transform)

test_dataset = torchvision.datasets.CIFAR10(root = "./data", train = False,
                                            download = True,
                                            transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [25]:
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                          batch_size = batch_size,
                                          shuffle = True)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size = batch_size,
                                          shuffle = True)

In [26]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog',
           'horse', 'ship', 'truck']

In [51]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x

In [52]:
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [53]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 62,006 trainable parameters


In [54]:
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        # forward
        predicts = model(images)
        
        # loss
        loss = criterion(predicts, labels)
        
        # gradients
        loss.backward()
        
        # update weights
        optimizer.step()
        optimizer.zero_grad()
        
        if (i+1)%2_000 == 0:
            print(f"epoch: {epoch}/{num_epochs}, step: {i+1}/{n_total_steps}, loss: {loss.item():.3f}")

print("Finish training")


with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(1, batch_size//2):
            label = labels[i-1]
            pred = predicted[i-1]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Finish training
Accuracy of the network: 54.2 %
Accuracy of plane: 51.52838427947598 %
Accuracy of car: 71.06382978723404 %
Accuracy of bird: 38.983050847457626 %
Accuracy of cat: 36.38344226579521 %
Accuracy of deer: 28.043478260869566 %
Accuracy of dog: 54.716981132075475 %
Accuracy of frog: 74.77876106194691 %
Accuracy of horse: 55.93952483801296 %
Accuracy of ship: 69.95967741935483 %
Accuracy of truck: 60.040983606557376 %
